## Imports

In [60]:
# IMPORTS
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras import layers
import random
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.operators.sampling.rnd import BinaryRandomSampling
from pymoo.operators.crossover.pntx import TwoPointCrossover
from pymoo.operators.mutation.bitflip import BitflipMutation
from pymoo.operators.selection.tournament import TournamentSelection
from pymoo.termination.default import DefaultMultiObjectiveTermination
from tensorflow.keras.models import clone_model
from pymoo.core.problem import Problem
from keras.utils import to_categorical


## Classes

In [61]:
# CLASSES

class Server:
    def __init__(self, devices_list):
        self.model = Server.create_model()
        self.current_learning_iteration = 0
        self.LAST_WEIGHTS_SENT_FOR_ALL_DEVICES = []
        self.x_test_global = []
        self.y_test_global = []
        self.devices = devices_list

    # def evaluate(self, verbose = 1):
    #     test_loss, test_acc = self.model.evaluate(self.x_test_global, self.y_test_global, verbose)
    #     return test_loss, test_acc
    
    def evaluate(self, x_test=None, y_test=None, verbose = 1):
        if x_test is None and y_test is None:
            test_loss, test_acc = self.model.evaluate(self.x_test_global, self.y_test_global, verbose)
            return test_loss, test_acc
        test_loss, test_acc = self.model.evaluate(x_test, y_test, verbose=verbose)
        return test_loss, test_acc
    
    def get_weights(self):
        return self.model.get_weights()

    def set_aggregated_weight(self):
        self.model.set_weight(Server.aggregate_weights())
    
    def give_global_model_weights_to_bitstring_devices(self, bitstring):
        for device in self.devices:
            if int(bitstring[int(device.id)]) == 1:
                device.model.set_weights(self.model.get_weights())

    def create_model():
        model = keras.Sequential([
            layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dense(128, activation='relu'),
            layers.Dense(10, activation='softmax')
        ])
        model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.01),
                        # new
                        loss='categorical_crossentropy', metrics=['accuracy'])
        return model
    
    def aggregate_weights(self, bitstring):
        """Computes the weighted average of model weights from all devices and updates the global model."""
        def sum_all_nested_lists(list_of_lists):
            def recursive_sum(lists):
                if isinstance(lists[0], list):
                    return [recursive_sum([lst[i] for lst in lists]) for i in range(len(lists[0]))]
                else:
                    return sum(lists)
            
            return recursive_sum(list_of_lists)

        def multiply_nested_list(self, lst, factor):
            result = []
            for item in lst:
                if isinstance(item, list):
                    # Recursively handle sublists
                    result.append(self.multiply_nested_list(item, factor))
                else:
                    # Multiply number
                    result.append(item * factor)
            return result
        
        selected_devices = []
        for device in self.devices:
            if int(bitstring[int(device.id)]) == 1:
                selected_devices.append(device)
        
        num_devices = len(selected_devices)
        if num_devices == 0:
            print("No devices available for aggregation.")
            return

        device_weights_all_layers = []
        device_participation_ratio = []
        data_lengths = []

        for device in selected_devices:
            device_weights_all_layers.append(device.model.get_weights())
            print("*******************")
            print(device.id)
            device_participation_ratio.append(device.last_round_participated / self.current_learning_iteration)
            print("this device's participation ratio:")
            print(device.last_round_participated / self.current_learning_iteration)
            data_lengths.append(len(device.data[0]))
            print("this device's data to all ratio:")
            print(len(device.data[0])/60000.0)

        sum_data = 0
        for data_len in data_lengths:
            sum_data += data_len
        
        data_fractions = []
        for device in selected_devices:
            data_fractions.append(len(device.data[0])/float(sum_data))

        aggregated_weights_devices = []
        for d in range(len(selected_devices)):
            aggregated_weights_devices.append(multiply_nested_list(self, selected_devices[d].model.get_weights(), data_fractions[d]*device_participation_ratio[d]))
        
        aggregated_weights = sum_all_nested_lists(aggregated_weights_devices)
        # TODO: Weighted multiplication for each node in each layer of the neural network of the received devices and then summing
        #       the related parts together so that we get a full weighted average of all these devices' models

        print("Aggregated weights:")
        for layer_idx, layer_weights in enumerate(aggregated_weights):
            print(f"Layer {layer_idx}: {layer_weights.shape}")
        return aggregated_weights


class Device:
    def __init__(self, id, ram, storage, cpu, bandwidth, battery, charging):
        self.id = id
        self.ram = ram
        self.storage = storage
        self.cpu = cpu
        self.bandwidth = bandwidth
        self.battery = battery
        self.charging = charging
        self.model = Server.create_model()
        self.last_round_participated = 0
        self.data = None  # Placeholder for dataset partition
        self.test_data = None
        self.number_of_times_fitted = 0


class FederatedLearningProblem(Problem):
    def __init__(self, devices, server: Server):
        super().__init__(
            n_var=len(devices),         # Number of variables (bitstring length)
            n_obj=3,                   # Number of objectives
            n_constr=0,                # No constraints
            xl=np.zeros(len(devices)),  # Lower bound (0)
            xu=np.ones(len(devices)),   # Upper bound (1)
            type_var=np.bool_          # Binary variables (bitstrings)
        )
        self.devices = devices
        self.server = server
        self.x_test_global = server.x_test_global
        self.y_test_global = server.y_test_global

        # Save the initial global model weights
        self.initial_global_weights = server.get_weights()

    def _evaluate(self, X, out, *args, **kwargs):
        """Evaluates objective values for each solution in the population."""
        num_solutions = len(X)
        F = np.zeros((num_solutions, 3))  # Initialize objective matrix

        for i, bitstring in enumerate(X):
            print(f"evaluating: {bitstring}")
            # TODO: check bitstring type
            # Reset the global model to its initial state
            # Update device participation based on the bitstring
            selected_devices = [device for device, bit in zip(self.devices, bitstring) if int(bit) == 1]
            
            # Objective 1: Hardware Objectives (maximize)
            hardware_score = 0.0
            for device in selected_devices:
                device_hardware_score = float(6 - (device.ram + device.storage + device.cpu + device.bandwidth + device.battery + device.charging)) / 6.0
                hardware_score += device_hardware_score

            F[i, 0] = hardware_score  # Minimize (negative of hardware score)

            fairness_score = 0
            for device in self.devices:
                if bitstring[int(device.id)] == 1:
                    # new
                    _, accuracy = self.server.evaluate(device.test_data[0], device.test_data[1], verbose=0)
                    fairness_score += accuracy

            F[i, 1] = fairness_score/float(len(selected_devices))  # Minimize (negative of fairness score)  # Added (/Selected Devices) to normalize between 0 and 1
            
            # Objective 3: Global Model Accuracy (Performance) (maximize)
            temp_global_model = Server.create_model()
            # new
            temp_global_model.set_weights(self.performance_objective_aggregation(selected_devices))
            _, global_accuracy = temp_global_model.evaluate(self.server.x_test_global, self.server.y_test_global, verbose=0)
            F[i, 2] = 1 - global_accuracy  # Minimize (1 - accuracy)
            
        out["F"] = F  # Set the objective values

    def performance_objective_aggregation(self, selected_devices):

        def sum_all_nested_lists(list_of_lists):
            def recursive_sum(lists):
                if isinstance(lists[0], list):
                    return [recursive_sum([lst[i] for lst in lists]) for i in range(len(lists[0]))]
                else:
                    return sum(lists)
            
            return recursive_sum(list_of_lists)

        def multiply_nested_list(lst, factor):
            result = []
            for item in lst:
                if isinstance(item, list):
                    # Recursively handle sublists
                    result.append(multiply_nested_list(item, factor))
                else:
                    # Multiply number
                    result.append(item * factor)
            return result

        num_devices = len(selected_devices)
        if num_devices == 0:
            print("No devices available for aggregation.")
            return

        device_weights_all_layers = []
        device_participation_ratio = []
        data_lengths = []

        for device in selected_devices:
            device_weights_all_layers.append(self.server.LAST_WEIGHTS_SENT_FOR_ALL_DEVICES[int(device.id)])
            # print("*******************")
            # print(device.id)
            device_participation_ratio.append(device.last_round_participated / self.server.current_learning_iteration)
            # print("this device's participation ratio:")
            # print(device.last_round_participated / self.server.current_learning_iteration)
            
            data_lengths.append(len(device.data[0]))
            # print("this device's data to all ratio:")
            # print(len(device.data[0])/60000.0)

        sum_data = 0
        for data_len in data_lengths:
            sum_data += data_len
        
        data_fractions = []
        for device in selected_devices:
            data_fractions.append(len(device.data[0])/float(sum_data))

        aggregated_weights_devices = []
        for d in range(len(selected_devices)):
            aggregated_weights_devices.append(multiply_nested_list(self.server.LAST_WEIGHTS_SENT_FOR_ALL_DEVICES[int(selected_devices[d].id)], data_fractions[d]*device_participation_ratio[d]))
        
        aggregated_weights = sum_all_nested_lists(aggregated_weights_devices)
        # TODO: Weighted multiplication for each node in each layer of the neural network of the received devices and then summing
        #       the related parts together so that we get a full weighted average of all these devices' models

        # print("Aggregated weights:")
        # for layer_idx, layer_weights in enumerate(aggregated_weights):
        #     print(f"Layer {layer_idx}: {layer_weights.shape}")
        return aggregated_weights
    



## Functions

In [62]:
# Functions

def fit_bitstring_devices(bitstring, server: Server):
    '''
    server: for using its "current_learning_iteration" variable
    '''
    print(1)
    # global current_learning_iteration
    # global LAST_WEIGHTS_SENT_FOR_ALL_DEVICES
    server.current_learning_iteration += 1
    for device in server.devices:
        if bitstring[int(device.id)] == 1:
            device.model.fit(device.data[0], device.data[1], epochs=7, verbose=1)
            print(device.id)
            device.last_round_participated = server.current_learning_iteration
            server.LAST_WEIGHTS_SENT_FOR_ALL_DEVICES[int(device.id)] = device.model.get_weights()
            device.number_of_times_fitted += 1



## Load Data

### Load Devices

In [63]:
# Load dataset from CSV
csv_file = 'devices.csv'
df = pd.read_csv(csv_file)
df.columns = df.columns.str.strip().str.lower()

# Convert CSV rows into device objects
devices = []

for _, row in df.iterrows():
    device = Device(
        row['id'], row['ram'], row['storage'], row['cpu'], row['bandwidth'], row['battery'],
        row.get('charging', 0)
    )
    devices.append(device)


# LIMIT TO 30 DEVICES
devices = devices[:30]

d:\github project\FL\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Object Initializations

In [64]:
# Global Model
server = Server(devices_list=devices)
server.LAST_WEIGHTS_SENT_FOR_ALL_DEVICES = [None for _ in range(len(devices))]

### Split Data Among Devices

In [65]:
# Load MNIST dataset  # new
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Convert labels to categorical (one-hot encoded)
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Normalize data and reshape for CNN
x_train = x_train.astype("float32") / 255.0
x_train = np.expand_dims(x_train, -1)  # Add channel dimension

x_test = x_test.astype("float32") / 255.0
x_test = np.expand_dims(x_test, -1)  # Add channel dimension

# Shuffle data
# indices = np.arange(len(x_train))
# np.random.shuffle(indices)
# x_train, y_train = x_train[indices], y_train[indices]

# Correct test split
split_index = int(0.8 * len(x_test))
x_test_devices, y_test_devices = x_test[:split_index], y_test[:split_index]
server.x_test_global, server.y_test_global = x_test[split_index:], y_test[split_index:]

# Training data (for devices)
x_train_devices, y_train_devices = x_train, y_train

# Split training data among devices
num_devices = len(devices)
split_size = len(x_train_devices) // num_devices

for i, device in enumerate(devices):
    start = i * split_size
    end = (i + 1) * split_size if i < num_devices - 1 else len(x_train_devices)
    device.data = (x_train_devices[start:end], y_train_devices[start:end])

# Split test data (device-level)
split_size = len(x_test_devices) // num_devices

for i, device in enumerate(devices):
    start = i * split_size
    end = (i + 1) * split_size if i < num_devices - 1 else len(x_test_devices)
    device.test_data = (x_test_devices[start:end], y_test_devices[start:end])

### Load Other Data

## First Iteration

In [66]:
# First Iteration
bitstring = [1 for _ in range(len(devices))]
print(bitstring)

# global model sends its weights to all devices
server.give_global_model_weights_to_bitstring_devices(bitstring)

test_loss, test_acc = server.evaluate(verbose=0)
print(f"Global Model Accuracy: {test_acc:.4f}")
print("------------------------------------------------------------")
fit_bitstring_devices(bitstring, server)
server.model.set_weights(server.aggregate_weights(bitstring))
print("------------------------------------------------------------")
test_loss, test_acc = server.evaluate(verbose=0)
print(f"Global Model Accuracy: {test_acc:.4f}")

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.1272 - loss: 2.2839
Global Model Accuracy: 0.1185
------------------------------------------------------------
1
Epoch 1/7
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.2806 - loss: 2.2302
Epoch 2/7
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.6950 - loss: 1.7668
Epoch 3/7
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7853 - loss: 1.0465
Epoch 4/7
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.8546 - loss: 0.6625
Epoch 5/7
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8708 - loss: 0.5021
Epoch 6/7
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8745 - loss: 0.4515
Epoch 7/7
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8881 - loss: 0.4129
0.0
Epoch 1/7
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.3065 - loss: 2.2268
Epoch 2/7
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.704

In [67]:
!pip install pymoo


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## NSGA2 Loop

In [ ]:
# Parameters
POPULATION_SIZE = 50
NUM_GENERATIONS = 3
NUM_ROUNDS = 1

In [ ]:
# Object Initializations

problem = FederatedLearningProblem(
    devices=devices,
    server=server
)


# Step 2: Configure NSGA-II Algorithm
algorithm = NSGA2(
    pop_size=POPULATION_SIZE,
    sampling=BinaryRandomSampling(),      # Random bitstrings
    crossover=TwoPointCrossover(),        # Two-point crossover
    mutation=BitflipMutation(),           # Bit flip mutation
    eliminate_duplicates=True             # Avoid duplicate solutions
)

server.current_learning_iteration += 1

In [70]:
# DEBUG:
print(server.x_test_global.shape)
print(server.y_test_global.shape)

print(x_train.shape)
print(y_train.shape)

(2000, 28, 28, 1)
(2000, 10)
(60000, 28, 28, 1)
(60000, 10)


In [ ]:
for i in range(NUM_ROUNDS):
    # Step 3: Run Optimization
    print("GLOBAL MODEL BEFORE OPTIMIZATION")
    print(server.evaluate())
    res = minimize(
        problem=problem,
        algorithm=algorithm,
        termination=DefaultMultiObjectiveTermination(n_max_gen=NUM_GENERATIONS),
        # seed=42,
        verbose=True
    )
    print("GLOBAL MODEL AFTER OPTIMIZATION")
    print(server.evaluate())


    # Step 4: Extract the Best Pareto Front
    pareto_front = res.F   # Objective values of solutions in Pareto front
    pareto_solutions = res.X  # Corresponding bitstrings

    # Print the Best Pareto Front Solutions
    print("Best Pareto Front (Bitstrings):")
    for bitstring in pareto_solutions:
        print("".join(map(str, bitstring)).replace('True','1').replace('False','0'))

    bitstring = pareto_solutions[0] # for now!
    bitstring = str(bitstring).replace('False','0').replace('True','1')
    for char in bitstring:
        if char != '0' and char != '1':
            bitstring = bitstring.replace(char,'')

    print(len(bitstring))
    print(bitstring)
    temp_bitstring = []
    for bit in bitstring:
        temp_bitstring.append(bit)
    bitstring = temp_bitstring


    test_loss, test_acc = server.evaluate()
    print(f"Global Model Accuracy: {test_acc:.4f}")
    print("------------------------------------------------------------")
    server.give_global_model_weights_to_bitstring_devices(bitstring)
    fit_bitstring_devices(bitstring)
    server.model.set_weights(server.aggregate_weights(bitstring))
    print("------------------------------------------------------------")
    test_loss, test_acc = server.evaluate()
    print(f"Global Model Accuracy: {test_acc:.4f}")
    

GLOBAL MODEL BEFORE OPTIMIZATION
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 27s 13ms/step - accuracy: 0.9553 - loss: 0.2261
(0.2743990123271942, 0.9390000104904175)
evaluating: [False False  True  True False  True  True False  True False  True False
 False  True  True False False False False False  True False  True  True
 False False False False  True False]
evaluating: [ True False  True False False  True False False  True  True  True False
 False  True  True False False False  True  True  True  True False False
 False  True  True False  True  True]
evaluating: [ True  True  True  True  True False  True False False False  True  True
 False False  True False False  True False  True False  True  True  True
 False False  True  True  True False]
evaluating: [False  True  True False False False False False  True  True False False
 False False  True  True  True  True False  True False False  True  True
 False False False  True False False]
evaluating: [ True  True  True  True False False False  True Fa

KeyboardInterrupt: 

In [ ]:
# Step 4: Extract the Best Pareto Front
pareto_front = res.F   # Objective values of solutions in Pareto front
pareto_solutions = res.X  # Corresponding bitstrings

# Print the Best Pareto Front Solutions
print("Best Pareto Front (Bitstrings):")
for bitstring in pareto_solutions:
    print("".join(map(str, bitstring)).replace('True','1').replace('False','0'))

bitstring = pareto_solutions[0] # for now!
bitstring = str(bitstring).replace('False','0').replace('True','1')
for char in bitstring:
    if char != '0' and char != '1':
        bitstring = bitstring.replace(char,'')

print(len(bitstring))
print(bitstring)
temp_bitstring = []
for bit in bitstring:
    temp_bitstring.append(bit)
bitstring = temp_bitstring

########################################################
# Update device participation based on the bitstring
selected_devices = [device for device in devices if bitstring[int(device.id)] == '1']
# give_global_model_weights_to_bitstring_devices(bitstring)
fit_bitstring_devices(bitstring)

# new
test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
print(f"Global Model Accuracy: {test_acc:.4f}")
print("------------------------------------------------------------")
global_model.set_weights(aggregate_weights(bitstring))
print("------------------------------------------------------------")
test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
print(f"Global Model Accuracy: {test_acc:.4f}")

Best Pareto Front (Bitstrings):
010010000000001110001100011101
001100100000111000011010001110
000000100000001110001100011101
100100011001100010010010000000
010100011010100000111000010000
010010001110100010001001000000
110000100000001110001100101101
100010101000100001110001101000
000001000000000110001001000000
000100100000100010011010001110
000100000000001000111110001011
111010000000100100001110000101
30
010010000000001110001100011101
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1128 - loss: 2.3028
Global Model Accuracy: 0.1235
------------------------------------------------------------
*******************
1.0
this device's participation ratio:
0.2857142857142857
this device's data to all ratio:
0.03333333333333333
*******************
4.0
this device's participation ratio:
0.2857142857142857
this device's data to all ratio:
0.03333333333333333
*******************
14.0
this device's participation rat

In [ ]:
print(bitstring) # for now!
print(len(bitstring))
print(bitstring.count("1"))

['0', '0', '0', '0', '1', '1', '1', '1', '0', '0', '1', '1', '0', '1', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '1', '0', '0', '0', '0', '1', '0', '1', '0', '0', '1', '0', '0', '0', '0', '0', '1', '1', '1', '0', '1', '1', '0', '0', '1', '1', '1', '1', '1', '1', '0', '0', '1', '0', '0', '1', '1', '0', '1', '0', '0', '0', '1', '0', '0', '1', '1', '0', '0', '1', '0', '1', '0', '0', '0', '1', '0', '0', '1', '0', '1', '0', '1', '1', '0', '1', '0', '0', '1', '0', '0', '0', '1', '0']
100
39


In [ ]:
# Step 3: Run Optimization
problem.initial_global_weights = global_model.get_weights()
res = minimize(
    problem=problem,
    algorithm=algorithm,
    termination=DefaultMultiObjectiveTermination(n_max_gen=NUM_GENERATIONS),
    # seed=42,
    verbose=True
)

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       50 |      5 |             - |             -
     2 |      100 |      6 |  0.0129361668 |         ideal
     3 |      150 |      5 |  0.0786670050 |         ideal


In [ ]:
print(awiodawo)

In [ ]:
########################################################
# Update device participation based on the bitstring
selected_devices = [device for device in devices if bitstring[int(device.id)] == '1']

# aggregate_weights(global_model, selected_devices)
# aggregate_weights(global_model, devices)

# new
global_model.set_weights(aggregate_weights(devices))

# Distribute the updated global model back to all devices
for device in devices:
    device.model.set_weights(global_model.get_weights())

current_learning_iteration += 1
# Train local models for selected devices
for device in selected_devices:
    device.number_of_times_fitted += 1
    device.last_round_participated = current_learning_iteration
    loss_before = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    device.model.fit(device.data[0], device.data[1], epochs=10, verbose=0)
    loss_after = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    print(f"Device {device.id}: Loss before: {loss_before}, Loss after: {loss_after}")

# Aggregate weights to update the global model
w1 = global_model.get_weights()

# aggregate_weights(global_model, selected_devices)

# new
print(global_model.get_weights())
print("------------------------------------------------------------")
global_model.set_weights(aggregate_weights(selected_devices))
print(global_model.get_weights())
print("------------------------------------------------------------")

w2 = global_model.get_weights()
print("global model stayed the same?")
print(np.array_equal(w1,w2))
test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
print(f"Global Model Accuracy: {test_acc:.4f}")

Device 4.0: Loss before: [2.2980031967163086, 0.14666666090488434], Loss after: [2.284519910812378, 0.35333332419395447]
Device 5.0: Loss before: [2.298715114593506, 0.11999999731779099], Loss after: [2.2860357761383057, 0.34333333373069763]
Device 6.0: Loss before: [2.298321008682251, 0.14166666567325592], Loss after: [2.2865428924560547, 0.4116666615009308]
Device 7.0: Loss before: [2.298210859298706, 0.13833333551883698], Loss after: [2.282723903656006, 0.22833333909511566]
Device 10.0: Loss before: [2.2980704307556152, 0.1616666615009308], Loss after: [2.2842841148376465, 0.2266666740179062]
Device 11.0: Loss before: [2.2986857891082764, 0.13333334028720856], Loss after: [2.285165786743164, 0.22333332896232605]
Device 13.0: Loss before: [2.2982802391052246, 0.14499999582767487], Loss after: [2.2767813205718994, 0.1550000011920929]
Device 26.0: Loss before: [2.2987937927246094, 0.12166666984558105], Loss after: [2.286884307861328, 0.3266666531562805]
Device 31.0: Loss before: [2.298

In [ ]:
# Step 3: Run Optimization
problem.initial_global_weights = global_model.get_weights()
res = minimize(
    problem=problem,
    algorithm=algorithm,
    termination=DefaultMultiObjectiveTermination(n_max_gen=NUM_GENERATIONS),
    # seed=42,
    verbose=True
)########################################################
# Update device participation based on the bitstring
selected_devices = [device for device in devices if bitstring[int(device.id)] == '1']

# aggregate_weights(global_model, selected_devices)
# aggregate_weights(global_model, devices)

# new
global_model.set_weights(aggregate_weights(devices))

# Distribute the updated global model back to all devices
for device in devices:
    device.model.set_weights(global_model.get_weights())

current_learning_iteration += 1
# Train local models for selected devices
for device in selected_devices:
    device.number_of_times_fitted += 1
    device.last_round_participated = current_learning_iteration
    loss_before = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    device.model.fit(device.data[0], device.data[1], epochs=10, verbose=0)
    loss_after = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    print(f"Device {device.id}: Loss before: {loss_before}, Loss after: {loss_after}")

# Aggregate weights to update the global model
w1 = global_model.get_weights()

# aggregate_weights(global_model, selected_devices)

# new
print(global_model.get_weights())
print("------------------------------------------------------------")
global_model.set_weights(aggregate_weights(selected_devices))
print(global_model.get_weights())
print("------------------------------------------------------------")


w2 = global_model.get_weights()
print("global model stayed the same?")
print(np.array_equal(w1,w2))
test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
print(f"Global Model Accuracy: {test_acc:.4f}")

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       50 |      4 |             - |             -
     2 |      100 |      6 |  0.2727559400 |         ideal
     3 |      150 |      9 |  0.0710064480 |         ideal
Device 4.0: Loss before: [2.299807071685791, 0.12333333492279053], Loss after: [2.2950029373168945, 0.12333333492279053]
Device 5.0: Loss before: [2.300266981124878, 0.10833333432674408], Loss after: [2.295975685119629, 0.20333333313465118]
Device 6.0: Loss before: [2.3000829219818115, 0.1133333370089531], Loss after: [2.296647548675537, 0.1133333370089531]
Device 7.0: Loss before: [2.299997329711914, 0.10333333164453506], Loss after: [2.294761896133423, 0.13500000536441803]
Device 10.0: Loss before: [2.2999212741851807, 0.11666666716337204], Loss after: [2.2957682609558105, 0.15000000596046448]
Device 11.0: Loss before: [2.3003227710723877, 0.1133333370089531], Loss after: [2.296252965927124, 0.2266666740179062]
Device 13.0: Loss before: [2.299876928329

In [ ]:
# Step 3: Run Optimization
problem.initial_global_weights = global_model.get_weights()
res = minimize(
    problem=problem,
    algorithm=algorithm,
    termination=DefaultMultiObjectiveTermination(n_max_gen=NUM_GENERATIONS),
    # seed=42,
    verbose=True
)########################################################
# Update device participation based on the bitstring
selected_devices = [device for device in devices if bitstring[int(device.id)] == '1']

# aggregate_weights(global_model, selected_devices)
# aggregate_weights(global_model, devices)

# new
global_model.set_weights(aggregate_weights(devices))

# Distribute the updated global model back to all devices
for device in devices:
    device.model.set_weights(global_model.get_weights())

current_learning_iteration += 1
# Train local models for selected devices
for device in selected_devices:
    device.number_of_times_fitted += 1
    device.last_round_participated = current_learning_iteration
    loss_before = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    device.model.fit(device.data[0], device.data[1], epochs=10, verbose=0)
    loss_after = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    print(f"Device {device.id}: Loss before: {loss_before}, Loss after: {loss_after}")

# Aggregate weights to update the global model
w1 = global_model.get_weights()

# aggregate_weights(global_model, selected_devices)

# new
print(global_model.get_weights())
print("------------------------------------------------------------")
global_model.set_weights(aggregate_weights(global_model, selected_devices))
print(global_model.get_weights())
print("------------------------------------------------------------")


w2 = global_model.get_weights()
print("global model stayed the same?")
print(np.array_equal(w1,w2))
test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
print(f"Global Model Accuracy: {test_acc:.4f}")

n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |       50 |      5 |             - |             -
     2 |      100 |      6 |  0.0564924725 |         ideal
     3 |      150 |      8 |  0.4483626535 |         ideal
Device 4.0: Loss before: [2.3013622760772705, 0.12333333492279053], Loss after: [2.298837661743164, 0.12333333492279053]
Device 5.0: Loss before: [2.3016862869262695, 0.10833333432674408], Loss after: [2.2995495796203613, 0.10833333432674408]
Device 6.0: Loss before: [2.3016045093536377, 0.1133333370089531], Loss after: [2.300372362136841, 0.1133333370089531]
Device 7.0: Loss before: [2.3015823364257812, 0.10333333164453506], Loss after: [2.2990756034851074, 0.12166666984558105]
Device 10.0: Loss before: [2.3015029430389404, 0.11666666716337204], Loss after: [2.299830198287964, 0.11666666716337204]
Device 11.0: Loss before: [2.3018038272857666, 0.1133333370089531], Loss after: [2.300283908843994, 0.1133333370089531]
Device 13.0: Loss before: [2.301440000

TypeError: aggregate_weights() takes 1 positional argument but 2 were given

In [ ]:
# Step 3: Run Optimization
problem.initial_global_weights = global_model.get_weights()
res = minimize(
    problem=problem,
    algorithm=algorithm,
    termination=DefaultMultiObjectiveTermination(n_max_gen=NUM_GENERATIONS),
    # seed=42,
    verbose=True
)########################################################
# Update device participation based on the bitstring
selected_devices = [device for device in devices if bitstring[int(device.id)] == '1']

# aggregate_weights(global_model, selected_devices)
# aggregate_weights(global_model, devices)

# new
global_model.set_weights(aggregate_weights(global_model, devices))

# Distribute the updated global model back to all devices
for device in devices:
    device.model.set_weights(global_model.get_weights())

current_learning_iteration += 1
# Train local models for selected devices
for device in selected_devices:
    device.number_of_times_fitted += 1
    device.last_round_participated = current_learning_iteration
    loss_before = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    device.model.fit(device.data[0], device.data[1], epochs=10, verbose=0)
    loss_after = device.model.evaluate(device.data[0], device.data[1], verbose=0)
    print(f"Device {device.id}: Loss before: {loss_before}, Loss after: {loss_after}")

# Aggregate weights to update the global model
w1 = global_model.get_weights()

# aggregate_weights(global_model, selected_devices)

# new
print(global_model.get_weights())
print("------------------------------------------------------------")
global_model.set_weights(aggregate_weights(selected_devices))
print(global_model.get_weights())
print("------------------------------------------------------------")


w2 = global_model.get_weights()
print("global model stayed the same?")
print(np.array_equal(w1,w2))
test_loss, test_acc = global_model.evaluate(x_test_global, y_test_global)
print(f"Global Model Accuracy: {test_acc:.4f}")